In [2]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict

In [3]:
class BatsmanState(TypedDict):
    runs:int
    balls:int
    fours:int
    sixes:int
    
    sr: float # strike rate
    bpb:float # balls per boundary
    boundary_percent: float
    summary:str

In [4]:
#function for calculate_sr node 
def calculate_sr(state:BatsmanState):
    sr = (state['runs']/state['balls'])*100
    
    return {'sr':sr}

In [5]:
#function for calculate_bpb node 
def calculate_bpb(state:BatsmanState):
    bpb = round(state['balls']/(state['fours']+state['sixes']),2)
    
    return {'bpb':bpb}

In [6]:
#function for calculate_boundary_percent node 

def calculate_boundary_percent(state:BatsmanState):
    bp = (((state['fours']*4) + (state['sixes']*6))/state['runs'])*100

    return {'boundary_percent':bp}

In [7]:
def summary(state:BatsmanState)->BatsmanState:
    summary = f"""
    Strike rate - {state['sr']} \n
    Balls per boundary - {state['bpb']} \n
    Boundary percent - {state['boundary_percent']}
    """
    
    return {'summary':summary}

In [8]:
graph = StateGraph(BatsmanState)

# nodes
graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundary_percent',calculate_boundary_percent)
graph.add_node('summary',summary)

# edges

graph.add_edge(START,"calculate_sr")
graph.add_edge(START,"calculate_bpb")
graph.add_edge(START,"calculate_boundary_percent")

graph.add_edge("calculate_sr","summary")
graph.add_edge("calculate_bpb","summary")
graph.add_edge("calculate_boundary_percent","summary")

graph.add_edge("summary",END)

workflow = graph.compile()



In [9]:
initial_state = {
    'runs': 100,
    'balls':50,
    'fours':6,
    'sixes':4
}

final_state = workflow.invoke(initial_state)

In [10]:
print(final_state)

{'runs': 100, 'balls': 50, 'fours': 6, 'sixes': 4, 'sr': 200.0, 'bpb': 5.0, 'boundary_percent': 48.0, 'summary': '\n    Strike rate - 200.0 \n\n    Balls per boundary - 5.0 \n\n    Boundary percent - 48.0\n    '}
